In [ ]:
import uproot as up
import numpy as np
import time
import iminuit
from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
from iminuit.cost import UnbinnedNLL
from scipy.optimize import minimize
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
from multiprocessing import Pool
import multiprocessing

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

sys.path.append('/software/pc24403/tfpcbpggsz/amp_ampgen')
from D0ToKSpipi2018 import *

Kspipi = PyD0ToKSpipi2018()
Kspipi.init()
print('INFO: Loading the amplitude model')




# Mass shape

In [ ]:
sys.path.append('/software/pc24403/tfpcbpggsz/func')
from tfmassshape import *

In [ ]:
from importlib.machinery import SourceFileLoader

config_mass_shape_input = SourceFileLoader('config_mass_shape_input', '/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/config/%s'%('config_CPrange_input_2.py')).load_module()
varDict_init = config_mass_shape_input.getconfig()


In [ ]:
def get_mass(p1,p2):
    return ((p1[:,0]+p2[:,0])**2 - (p1[:,1]+p2[:,1])**2 - (p1[:,2]+p2[:,2])**2 - (p1[:,3]+p2[:,3])**2)


def get_p4(decay="b2dpi", cut='', index=2):

    file_name = ''
    branch_names = []
    if cut == 'int':
        file_name = f'/software/pc24403/PCBPGGSZ/Int/weighted_{decay}.root:DalitzEventList'
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
    
    elif decay.split('_')[0] == 'b2dk' or decay.split('_')[0] == 'b2dpi':
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
        if cut == 'p':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/add_sw/new_frac_sw_pg/lhcb_toy_{decay}_{index}_CPrange.root:Bplus_DalitzEventList'
            #file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x_test/{decay}_sig_{index}.root:Bplus_DalitzEventList'

        else:
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/add_sw/new_frac_sw_pg/lhcb_toy_{decay}_{index}_CPrange.root:Bminus_DalitzEventList'
            #file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/1x_test/{decay}_sig_{index}.root:Bminus_DalitzEventList'

    tree = up.open(file_name)
  # Load the branches as arrays
    
    array = tree.arrays(branch_names)
       

    _p1 = np.asarray([array["_1_K0S0_E"], array["_1_K0S0_Px"], array["_1_K0S0_Py"], array["_1_K0S0_Pz"]])
    _p2 = np.asarray([array["_2_pi#_E"], array["_2_pi#_Px"], array["_2_pi#_Py"], array["_2_pi#_Pz"]])
    _p3 = np.asarray([array["_3_pi~_E"], array["_3_pi~_Px"], array["_3_pi~_Py"], array["_3_pi~_Pz"]])
    
    # convert 4*1000 into a vectot<double>
    p1 = np.transpose(_p1)
    p2 = np.transpose(_p2)
    p3 = np.transpose(_p3)

    p1bar = np.hstack((p1[:, :1], np.negative(p1[:, 1:])))
    p2bar = np.hstack((p2[:, :1], np.negative(p2[:, 1:])))
    p3bar = np.hstack((p3[:, :1], np.negative(p3[:, 1:])))



    return p1, p2, p3, p1bar, p2bar, p3bar

def load_int_amp(args):
    p1, p2, p3 = args

    return Kspipi.AMP(p1.tolist(), p2.tolist(), p3.tolist())

def getAmp(decay='b2dpi', cut='int'):

    start_time = time.time()
    p1, p2, p3, p1bar, p2bar, p3bar = get_p4(decay=decay, cut=cut)
    amplitude = []
    amplitudeBar = []

    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)
    p1bar_np = np.array(p1bar)
    p2bar_np = np.array(p2bar)
    p3bar_np = np.array(p3bar)

    data = [(p1_np[i], p2_np[i], p3_np[i]) for i in range(len(p1_np))]
    with Pool(processes=multiprocessing.cpu_count()) as pool:
        amplitude.append(pool.map(load_int_amp, data))
    data_bar = [(p1bar_np[i], p3bar_np[i], p2bar_np[i]) for i in range(len(p1bar_np))]
    with Pool(processes=multiprocessing.cpu_count()) as pool:
        amplitudeBar.append(pool.map(load_int_amp, data_bar))
    
    end_time = time.time()
    print(f'Amplitude for {decay} loaded in {end_time-start_time} seconds')
    amplitude = np.array(amplitude)
    amplitudeBar = np.array(amplitudeBar)

    return amplitude, amplitudeBar
    
def get_p4_v2(decay="b2dpi", cut='', index=2, comp='sig'):

    file_name = ''
    branch_names = []
    if cut == 'int':
        file_name = f'/software/pc24403/PCBPGGSZ/Int/weighted_{decay}.root:DalitzEventList'
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
    
    elif decay.split('_')[0] == 'b2dk' or decay.split('_')[0] == 'b2dpi':
        branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz", "B_M"]
        if cut == 'p':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/add_sw/new_frac_sw_pg/{decay}_{index}_CPrange.root:DalitzEventList'
#            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/swap/{decay}_{comp}_{index}.root:Bplus_DalitzEventList'

        else:
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/add_sw/new_frac_sw_pg/{decay}_{index}_CPrange.root:DalitzEventList'
#            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/swap/{decay}_{comp}_{index}.root:Bminus_DalitzEventList'

    tree = up.open(file_name)
  # Load the branches as arrays
    charge = '(Bac_ID>0)'
    if cut == 'm':
        charge = '(Bac_ID<0)'
    
    array = tree.arrays(branch_names, charge)
       

    _p1 = np.asarray([array["_1_K0S0_E"], array["_1_K0S0_Px"], array["_1_K0S0_Py"], array["_1_K0S0_Pz"]])
    _p2 = np.asarray([array["_2_pi#_E"], array["_2_pi#_Px"], array["_2_pi#_Py"], array["_2_pi#_Pz"]])
    _p3 = np.asarray([array["_3_pi~_E"], array["_3_pi~_Px"], array["_3_pi~_Py"], array["_3_pi~_Pz"]])
    
    # convert 4*1000 into a vectot<double>
    p1 = np.transpose(_p1)
    p2 = np.transpose(_p2)
    p3 = np.transpose(_p3)

    p1bar = np.hstack((p1[:, :1], np.negative(p1[:, 1:])))
    p2bar = np.hstack((p2[:, :1], np.negative(p2[:, 1:])))
    p3bar = np.hstack((p3[:, :1], np.negative(p3[:, 1:])))

    B_M = np.zeros(len(p1))
    if cut != 'int':
        
        B_M = np.asarray([array["B_M"]])


    return p1, p2, p3, p1bar, p2bar, p3bar, B_M


def getMass(decay='b2dpi', cut='int'):

    start_time = time.time()
    p1, p2, p3, p1bar, p2bar, p3bar = get_p4(decay=decay, cut=cut)
    amplitude = []
    amplitudeBar = []

    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)
    p1bar_np = np.array(p1bar)
    p2bar_np = np.array(p2bar)
    p3bar_np = np.array(p3bar)

    s12 = get_mass(p1_np, p2_np)
    s13 = get_mass(p1_np, p3_np)

    return s12, s13

def getMass_v2(decay='b2dpi', cut='int', comp='sig'):

    start_time = time.time()
    p1, p2, p3, p1bar, p2bar, p3bar, B_M = get_p4_v2(decay=decay, cut=cut, comp=comp)
    amplitude = []
    amplitudeBar = []

    p1_np = np.array(p1)
    p2_np = np.array(p2)
    p3_np = np.array(p3)
    p1bar_np = np.array(p1bar)
    p2bar_np = np.array(p2bar)
    p3bar_np = np.array(p3bar)

    s12 = get_mass(p1_np, p2_np)
    s13 = get_mass(p1_np, p3_np)

    return s12, s13, B_M

In [ ]:
sys.path.append('/software/pc24403/tfpcbpggsz/func')
sys.path.append('/software/pc24403/tfpcbpggsz/amp_ampgen')
#
mc_path = '/shared/scratch/pc24403/amp_ampgen'
#
amp_Data_dk_dd_p, ampbar_Data_dk_dd_p = getAmp('b2dk_DD', 'p')
amp_Data_dk_dd_m, ampbar_Data_dk_dd_m = getAmp('b2dk_DD', 'm')
#
#amp_Data_dpi_dd_p, ampbar_Data_dpi_dd_p = getAmp('b2dpi_DD', 'p')
#amp_Data_dpi_dd_m, ampbar_Data_dpi_dd_m = getAmp('b2dpi_DD', 'm')

In [ ]:
pdfs_data = {}
s12_data = {}
s13_data = {}
Bu_M = {}
for decay in ['b2dk_DD', 'b2dpi_DD']:
    for charge in ['p', 'm']:
        new_decay = decay + '_'+ charge
        print('--- INFO: Preparing pdfs for %s...'%new_decay)
        s12_data[new_decay], s13_data[new_decay], Bu_M[new_decay] = getMass_v2(decay, charge)
        pdfs_data[new_decay] = preparePdf_data(Bu_M[new_decay], varDict_init, decay)

    Bu_M[decay] = np.concatenate((Bu_M[decay+'_p'].flatten(), Bu_M[decay+'_m'].flatten()))
    pdfs_data[decay] = preparePdf_data(Bu_M[decay], varDict_init, decay)


In [ ]:
from importlib.machinery import SourceFileLoader

config_mass_shape_output = SourceFileLoader('config_mass_shape_output', '/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/config/lhcb/%s'%('config_cpfit_output_2.py')).load_module()
varDict = config_mass_shape_output.getconfig()
x_exp = (varDict['n_sig_DK_KsPiPi_DD'], varDict['n_misid_DK_KsPiPi_DD'], varDict['n_comb_DK_KsPiPi_DD'], varDict['n_low_DK_KsPiPi_DD'], varDict['n_low_misID_DK_KsPiPi_DD'], varDict['n_low_Bs2DKPi_DK_KsPiPi_DD'])


In [30]:
%%time
@tf.function
def nll_extended_dk_dd(par):
    decay = 'b2dk_DD'
    x = tf.cast(Bu_M[decay], tf.float64)
    nsig = tf.cast(par[0], tf.float64)
    nmisid = tf.cast(varDict['n_misid_DK_KsPiPi_DD'], tf.float64)
    ncomb = tf.cast(par[1], tf.float64)
    nlow = tf.cast(par[2], tf.float64)

    frac_low_misID = tf.cast(varDict['n_low_misID_DK_KsPiPi_DD']/varDict['n_low_DK_KsPiPi_DD'], tf.float64)
    nlow_misID = tf.cast(varDict['n_low_misID_DK_KsPiPi_DD'], tf.float64)
    nlow_Bs2DKPi = tf.cast(varDict['n_low_Bs2DKPi_DK_KsPiPi_DD'], tf.float64)

    pdf =  pdfs_data[decay]['sig'](x)*nsig
    pdf2 = pdfs_data[decay]['misid'](x)*nmisid
    pdf3 = pdfs_data[decay]['comb'](x)*ncomb
    pdf4 = pdfs_data[decay]['low'](x)*nlow
    pdf5 = pdfs_data[decay]['low_misID'](x)*nlow_misID
    pdf6 = pdfs_data[decay]['low_Bs2DKPi'](x)*nlow_Bs2DKPi
    ntotal = (nsig + nmisid + ncomb + nlow + nlow_misID + nlow_Bs2DKPi)

    npdf = pdf + pdf2 + pdf3 + pdf4 + pdf5 + pdf6

    nll = tf.reduce_sum(-2*clip_log(npdf)) + 2*ntotal
    return nll

@tf.function
def neg_like_and_gradient(parms):
    return tfp.math.value_and_gradient(nll_extended_dk_dd, parms)




CPU times: user 3.15 ms, sys: 35 µs, total: 3.19 ms
Wall time: 3.34 ms


In [28]:
x_ini = [varDict_init['cp_range_n_sig_DK_KsPiPi_DD']*0.5, varDict_init['cp_range_n_comb_DK_KsPiPi_DD']*0.5, varDict_init['cp_range_n_low_DK_KsPiPi_DD']*0.5]


In [29]:
m = Minuit(nll_extended_dk_dd, x_ini)
m.limits = [(0, varDict_init['cp_range_n_sig_DK_KsPiPi_DD']*3.5), (0, varDict_init['cp_range_n_comb_DK_KsPiPi_DD']*1.5), (10, varDict_init['cp_range_n_low_DK_KsPiPi_DD']*1.5)]
m.fixed = [False, False, False]
m.migrad()

Cause: Unable to locate the source code of <function nll_extended_dk_dd at 0x7fb174c81440>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function nll_extended_dk_dd at 0x7fb174c81440>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the fu

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.457e+05                 │              Nfcn = 93               │
│ EDM = 4.96e-05 (Goal: 0.0002)    │            time = 5.8 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  10.78e3  │  0.11e3   │            │            │    0    │ 33166.3 │       │
│ 1 │ x1   │  2.81e3   │  0.10e3   │            │            │    0    │ 4298.82 │       │
│ 2 │ x2   │    510    │    60     │            │            │   10    │ 779.317 │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌────┬───────────────────────────────┐
│    │        x0        x1        x2 │
├────┼───────────────────────────────┤
│ x0 │  1.31e+04 -1.96e+03       322 │
│ x1 │ -1.96e+03  9.83e+03 -2.52e+03 │
│ x2 │       322 -2.52e+03  4.21e+03 │
└────┴───────────────────────────────┘

In [21]:
print(x_exp)

(10706.312639596192, 4019.9590922912735, 2965.2445772918873, 434.39475280456827, 2515.0343065031257, 257.6967423637269)


In [38]:
Val = tf.constant([0., 0., 0., 0., 0., 0., varDict['n_sig_DK_KsPiPi_DD'], varDict['n_misid_DK_KsPiPi_DD'], varDict['n_comb_DK_KsPiPi_DD'], varDict['n_low_DK_KsPiPi_DD'], varDict['n_low_misID_DK_KsPiPi_DD'], varDict['n_low_Bs2DKPi_DK_KsPiPi_DD'], varDict['n_sig_DK_KsPiPi_LL'], varDict['n_misid_DK_KsPiPi_LL'], varDict['n_comb_DK_KsPiPi_LL'], varDict['n_low_DK_KsPiPi_LL'], varDict['n_low_misID_DK_KsPiPi_LL'], varDict['n_low_Bs2DKPi_DK_KsPiPi_LL'], varDict['n_misid_DPi_KsPiPi_DD'],  varDict['n_sig_DPi_KsPiPi_DD'], varDict['n_comb_DPi_KsPiPi_DD'], varDict['n_low_DPi_KsPiPi_DD'],  varDict['n_misid_DPi_KsPiPi_LL'], varDict['n_sig_DPi_KsPiPi_LL'], varDict['n_comb_DPi_KsPiPi_LL'], varDict['n_low_DPi_KsPiPi_LL']], shape=(26), dtype=tf.float64)

In [42]:
Val[12:18]

<tf.Tensor: shape=(6,), dtype=float64, numpy=
array([4690.60157469, 1819.13798458, 1438.49128096,  236.55952965,
       1106.10284797,  114.08363631])>

In [45]:
x1 = Val[:12]

x2 = tf.concat([Val[:6], Val[12:18]],0)

In [47]:
x1

<tf.Tensor: shape=(12,), dtype=float64, numpy=
array([    0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 10706.3126396 ,  4019.95909229,
        2965.24457729,   434.3947528 ,  2515.0343065 ,   257.69674236])>

In [ ]:
x_scan = np.linspace(2500, 3500, 10)

def nll_extended_dk_dd(par, x_int):
    decay = 'b2dk_DD'
    x = tf.cast(Bu_M[decay], tf.float64)
    nsig = tf.cast(varDict['n_sig_DK_KsPiPi_DD'], tf.float64)
    nmisid = tf.cast(varDict['n_misid_DK_KsPiPi_DD'], tf.float64)
    ncomb = tf.cast(x_int, tf.float64)
    nlow = tf.cast(par[2], tf.float64)

    frac_low_misID = tf.cast(varDict['n_low_misID_DK_KsPiPi_DD']/varDict['n_low_DK_KsPiPi_DD'], tf.float64)
    nlow_misID = tf.cast(varDict['n_low_misID_DK_KsPiPi_DD'], tf.float64)
    nlow_Bs2DKPi = tf.cast(varDict['n_low_Bs2DKPi_DK_KsPiPi_DD'], tf.float64)

    pdf =  pdfs_data[decay]['sig'](x)*nsig
    pdf2 = pdfs_data[decay]['misid'](x)*nmisid
    pdf3 = pdfs_data[decay]['comb'](x)*ncomb
    pdf4 = pdfs_data[decay]['low'](x)*nlow
    pdf5 = pdfs_data[decay]['low_misID'](x)*nlow_misID
    pdf6 = pdfs_data[decay]['low_Bs2DKPi'](x)*nlow_Bs2DKPi
    ntotal = (nsig + nmisid + ncomb + nlow + nlow_misID + nlow_Bs2DKPi)

    npdf = pdf + pdf2 + pdf3 + pdf4 + pdf5 + pdf6

    nll = tf.reduce_sum(-2*clip_log(npdf)) + 2*ntotal
    return nll

In [37]:
minimized_nsig = m.values[0]
minimized_nmisid = varDict_init['cp_range_n_misid_DK_KsPiPi_DD']#m.values[0]/varDict_init['pideff_DK_KsPiPi_k_to_k_DD'] * varDict_init['pideff_DK_KsPiPi_k_to_p_DD']
minimized_ncomb = m.values[1]
minimized_nlow = m.values[2]
minimized_nlow_misID = varDict_init['cp_range_n_low_misID_DK_KsPiPi_DD']
minimized_nlow_Bs2DKPi = varDict_init['cp_range_n_low_Bs2DKPi_DK_KsPiPi_DD']


In [33]:
# using the same values as above
testval = tf.constant([1.0, 2.0, 3.0], shape=(3), dtype=tf.float64)
out = neg_like_and_gradient(parms=testval)
print("Function value: ", out[0])
print("Gradients: ", out[1])

Function value:  tf.Tensor(-97291.58432279821, shape=(), dtype=float64)
Gradients:  tf.Tensor([-42.34302832  -8.9352982   -1.69631489], shape=(3,), dtype=float64)


In [34]:
# optimization
optim_results = tfp.optimizer.bfgs_minimize(
    neg_like_and_gradient, testval, tolerance=1e-8)


In [35]:
est_params = optim_results.position.numpy()
est_serr = np.sqrt(np.diagonal(optim_results.inverse_hessian_estimate.numpy()))
print("Estimated parameters: ", est_params)
print("Estimated standard errors: ", est_serr)

Estimated parameters:  [10781.81158461  2811.49745062   505.54593571]
Estimated standard errors:  [80.93448486 70.08974662 45.96015597]


In [36]:
out = neg_like_and_gradient(est_params)
print("Function value: ", out[0])
print("Gradients: ", out[1])


Function value:  tf.Tensor(-145694.24163969606, shape=(), dtype=float64)
Gradients:  tf.Tensor([-2.62874167e-11 -2.09086082e-11  1.15047971e-11], shape=(3,), dtype=float64)


In [ ]:


n_data = {'b2dk_DD': len(Bu_M['b2dk_DD']), 'b2dpi_DD': len(Bu_M['b2dpi_DD'])}
print(n_data)
bounds = [(-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD']), (-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD']), (-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD'])]
model = minimize(nll_extended_dk_dd, x_ini, method="L-BFGS-B", bounds=bounds)


In [ ]:
minimized_nsig = model.x[0]
minimized_nmisid = varDict_init['cp_range_n_misid_DK_KsPiPi_DD']#model.x[0]/varDict_init['pideff_DK_KsPiPi_k_to_k_DD'] * varDict_init['pideff_DK_KsPiPi_k_to_p_DD']
minimized_ncomb = model.x[1]
minimized_nlow = model.x[2]
minimized_nlow_misID = varDict_init['cp_range_n_low_misID_DK_KsPiPi_DD']
minimized_nlow_Bs2DKPi = varDict_init['cp_range_n_low_Bs2DKPi_DK_KsPiPi_DD']
model.success

In [ ]:
x_obs = [minimized_nsig, minimized_nmisid, minimized_ncomb, minimized_nlow, minimized_nlow_misID, minimized_nlow_Bs2DKPi]

In [ ]:
#Print the difference
print('nsig: %.2f'%minimized_nsig)
print('nmisid: %.2f'%minimized_nmisid)
print('ncomb: %.2f'%minimized_ncomb)
print('nlow: %.2f'%minimized_nlow)
print('nlow_misID: %.2f'%minimized_nlow_misID)
print('nlow_Bs2DKPi: %.2f'%minimized_nlow_Bs2DKPi)
print('ntotal: %.2f'%(minimized_nsig+minimized_nmisid+minimized_ncomb+minimized_nlow+minimized_nlow_misID+minimized_nlow_Bs2DKPi))
print('nsig: %.2f'%x_exp[0])
print('nmisid: %.2f'%x_exp[1])
print('ncomb: %.2f'%x_exp[2])
print('nlow: %.2f'%x_exp[3])
print('nlow_misID: %.2f'%x_exp[4])
print('nlow_Bs2DKPi: %.2f'%x_exp[5])
print('ntotal: %.2f'%(x_exp[0]+x_exp[1]+x_exp[2]+x_exp[3]+x_exp[4]+x_exp[5]))

In [ ]:
def plotOn(data=[], var=[], decay=decay,  nbins=100, ax=None, range=[5150, 5800]):


    density=False
    colors={'sig': 'blue', 'misid': 'yellowgreen', 'comb': 'violet', 'low': 'pink', 'low_misID': 'silver', 'low_Bs2DKPi': 'green'}
    filled={'sig': False, 'misid': False, 'comb': True, 'low': True, 'low_misID': True, 'low_Bs2DKPi': True}
    bin_edges = np.linspace(range[0], range[1], nbins+1)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    bin_centers = tf.cast(bin_centers, tf.float64)
    # Assuming s1 is sorted and binned accordingly, this is a placeholder for counting entries per bin
    counts, _ = np.histogram(data[decay], bins=bin_edges, density=density)
    # Calculate the standard deviation for the error bars
    std_devs = np.sqrt(counts)
    ax.errorbar(bin_centers, counts, yerr=std_devs, fmt='+', color='darkviolet',  label='Toy Data')
    #for key in prob.keys():
    #ax.hist(var[decay], bins=100, histtype='step', weights=weights['sig']*data[decay].shape[1], label='sig')
    #ax.hist([var[decay], var[decay], var[decay], var[decay], var[decay], var[decay]], bins=100, histtype='step', stacked=True, weights=[weights[key]*var[decay].shape[1] for key in weights.keys()], label=[key for key in weights.keys()])
    keys=[ 'misid', 'sig'] 
    if decay.split('_')[0] == 'b2dk':
        keys=['misid', 'sig']
        colors={'sig': 'yellowgreen', 'misid': 'blue', 'comb': 'violet', 'low': 'pink', 'low_misID': 'silver', 'low_Bs2DKPi': 'green'}
    weights = prob_extended(bin_centers, x_exp) 

    #ax.hist([var[decay] for key in keys], bins=100, histtype='step', stacked=False, weights=[(weights[key].flatten()) for key in keys], label=[key for key in keys], color=[colors[key] for key in keys], density=density)
    ax.plot(bin_centers, weights['total'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='red', label='Total ')
    ax.plot(bin_centers, weights['sig'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='blue', label='Signal')
    ax.plot(bin_centers, weights['misid'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='yellowgreen', label='MisID')
    ax.plot(bin_centers, weights['comb'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='violet', label='Combinatorial')
    ax.plot(bin_centers, weights['low'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='pink', label='Low')
    ax.plot(bin_centers, weights['low_misID'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='silver', label='Low MisID')
    ax.plot(bin_centers, weights['low_Bs2DKPi'](bin_centers)/sum(weights['total'](bin_centers))*len(data[decay]), color='green', label='Low Bs2DKPi')
    #ax.hist(var[decay], weights=sum_weights.flatten()*scale_factor, color='red', label='Total',bins=100, histtype='step', density=density)
    ax.legend()

    ax.set_xlabel('$M(B^{\pm})$')

In [ ]:
%%time


def prob_extended(Bu_M, par):
    decay = 'b2dk_DD'
    x = tf.cast(Bu_M, tf.float64)
    nsig = tf.cast(par[0], tf.float64)
    nmisid = tf.cast(par[1], tf.float64)
    ncomb = tf.cast(par[2], tf.float64)
    nlow = tf.cast(par[3], tf.float64)
    nlow_misID = tf.cast(par[4], tf.float64)
    nlow_Bs2DKPi = tf.cast(par[5], tf.float64)

    pdf = pdfs_data['b2dk_DD']['sig']
    pdf2 = pdfs_data['b2dk_DD']['misid']
    pdf3 = pdfs_data['b2dk_DD']['comb']
    pdf4 = pdfs_data['b2dk_DD']['low']
    pdf5 = pdfs_data['b2dk_DD']['low_misID']
    pdf6 = pdfs_data['b2dk_DD']['low_Bs2DKPi']
    ntotal = (nsig + nmisid + ncomb + nlow + nlow_misID + nlow_Bs2DKPi)

    npdf = nsig*pdf(x) + nmisid*pdf2(x) + ncomb*pdf3(x) + nlow*pdf4(x) + nlow_misID*pdf5(x) + nlow_Bs2DKPi*pdf6(x)

    prob={}
    prob['total'] = lambda x:  (nsig*pdf(x) + nmisid*pdf2(x) + ncomb*pdf3(x) + nlow*pdf4(x) + nlow_misID*pdf5(x) + nlow_Bs2DKPi*pdf6(x))/ntotal
    prob['sig'] = lambda x: nsig*pdf(x)/ntotal
    prob['misid'] = lambda x: nmisid*pdf2(x)/ntotal
    prob['comb'] = lambda x: ncomb*pdf3(x)/ntotal
    prob['low'] = lambda x: nlow*pdf4(x)/ntotal
    prob['low_misID'] = lambda x:nlow_misID*pdf5(x)/ntotal
    prob['low_Bs2DKPi'] = lambda x:nlow_Bs2DKPi*pdf6(x)/ntotal

    
    return prob


In [ ]:
fig, ax = plt.subplots()
plotOn(data=Bu_M, var=[], decay='b2dk_DD',  ax=ax, nbins=144)



# Only fit on DPi

In [ ]:
def nll_extended_dpi_dd(par):
    decay = 'b2dpi_DD'
    x= Bu_M[decay]
    nsig = par[0]
    nmisid = varDict_init['cp_range_n_sig_DK_KsPiPi_DD']/(varDict_init['pideff_DK_KsPiPi_k_to_k_DD']) * varDict_init['pideff_DK_KsPiPi_k_to_p_DD']
    ncomb = par[1]
    nlow = par[2]

    pdf = lambda x: pdfs_data[decay]['sig'](x)*nsig
    pdf2 = lambda x: pdfs_data[decay]['misid'](x)*nmisid
    pdf3 = lambda x: pdfs_data[decay]['comb'](x)*ncomb
    pdf4 = lambda x: pdfs_data[decay]['low'](x)*nlow

    ntotal = (nsig + nmisid + ncomb + nlow )

    npdf = pdf(x) + pdf2(x) + pdf3(x) + pdf4(x)

    nll = tf.reduce_sum(- 2 * np.log(npdf)) + 2 * ntotal
    
    return nll 

In [ ]:
x_ini = (varDict_init['n_sig_DPi_KsPiPi_DD'], varDict_init['n_comb_DPi_KsPiPi_DD'], varDict_init['n_low_DPi_KsPiPi_DD'])
bounds = [(-x_ini[0], 5*x_ini[0]), (-x_ini[1], 5*x_ini[1]), (-x_ini[2], 5*x_ini[2])]
model = minimize(nll_extended_dpi_dd, x_ini, method="L-BFGS-B", bounds=bounds)

In [ ]:
minimized_nsig = model.x[0]
minimized_nmisid = varDict_init['cp_range_n_misid_DPi_KsPiPi_DD']
minimized_ncomb = model.x[1]
minimized_nlow = model.x[2]


In [ ]:
x_exp = (varDict['n_sig_DPi_KsPiPi_DD'], varDict['n_misid_DPi_KsPiPi_DD'], varDict['n_comb_DPi_KsPiPi_DD'], varDict['n_low_DPi_KsPiPi_DD'])


In [ ]:
x_obs = [minimized_nsig, minimized_nmisid, minimized_ncomb, minimized_nlow]

In [ ]:
#Print the difference
print('nsig: %.2f'%minimized_nsig)
print('nmisid: %.2f'%minimized_nmisid)
print('ncomb: %.2f'%minimized_ncomb)
print('nlow: %.2f'%minimized_nlow)
print('ntotal: %.2f'%(minimized_nsig+minimized_nmisid+minimized_ncomb+minimized_nlow))
print('nsig: %.2f'%x_exp[0])
print('nmisid: %.2f'%x_exp[1])
print('ncomb: %.2f'%x_exp[2])
print('nlow: %.2f'%x_exp[3])
print('ntotal: %.2f'%(x_exp[0]+x_exp[1]+x_exp[2]+x_exp[3]))

In [ ]:
%%time


def prob_extended_dpi(Bu_M, par):
    decay = 'b2dpi_DD'
    x = Bu_M
    nsig = par[0]
    nmisid = par[1]
    ncomb = par[2]
    nlow = par[3]


    pdf = pdfs_data[decay]['sig']
    pdf2 = pdfs_data[decay]['misid']
    pdf3 = pdfs_data[decay]['comb']
    pdf4 = pdfs_data[decay]['low']
    ntotal = 1#(nsig + nmisid + ncomb + nlow)

    npdf = nsig*pdf(x) + nmisid*pdf2(x) + ncomb*pdf3(x) + nlow*pdf4(x)

    prob={}
    prob['total'] = lambda x:  (nsig*pdf(x) + nmisid*pdf2(x) + ncomb*pdf3(x) + nlow*pdf4(x) )/ntotal
    prob['sig'] = lambda x: nsig*pdf(x)/ntotal
    prob['misid'] = lambda x: nmisid*pdf2(x)/ntotal
    prob['comb'] = lambda x: ncomb*pdf3(x)/ntotal
    prob['low'] = lambda x: nlow*pdf4(x)/ntotal
    print(quad(prob['low'],5150, 5800)[0])

    
    return prob

def plotOn(data=[], var=[], decay=decay,  nbins=100, ax=None, range=[5150, 5800]):


    density=False
    colors={'sig': 'blue', 'misid': 'yellowgreen', 'comb': 'violet', 'low': 'pink', 'low_misID': 'silver', 'low_Bs2DKPi': 'green'}
    filled={'sig': False, 'misid': False, 'comb': True, 'low': True, 'low_misID': True, 'low_Bs2DKPi': True}
    bin_edges = np.linspace(range[0], range[1], nbins+1)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    # Assuming s1 is sorted and binned accordingly, this is a placeholder for counting entries per bin
    counts, _ = np.histogram(data[decay], bins=bin_edges, density=density)
    # Calculate the standard deviation for the error bars
    std_devs = np.sqrt(counts)
    ax.errorbar(bin_centers, counts, yerr=std_devs, fmt='+', color='darkviolet',  label='Toy Data')
    #for key in prob.keys():
    #ax.hist(var[decay], bins=100, histtype='step', weights=weights['sig']*data[decay].shape[1], label='sig')
    #ax.hist([var[decay], var[decay], var[decay], var[decay], var[decay], var[decay]], bins=100, histtype='step', stacked=True, weights=[weights[key]*var[decay].shape[1] for key in weights.keys()], label=[key for key in weights.keys()])
    keys=['misid', 'sig'] 
    if decay.split('_')[0] == 'b2dk':
        keys=['misid', 'sig']
        colors={'sig': 'yellowgreen', 'misid': 'blue', 'comb': 'violet', 'low': 'pink', 'low_misID': 'silver', 'low_Bs2DKPi': 'green'}

    weights = prob_extended_dpi(bin_centers, x_obs) 

    #ax.hist([var[decay] for key in keys], bins=100, histtype='step', stacked=False, weights=[(weights[key].flatten()) for key in keys], label=[key for key in keys], color=[colors[key] for key in keys], density=density)
    ax.plot(bin_centers, weights['total'](bin_centers).flatten()/sum(weights['total'](bin_centers))*len(data[decay]), color='red', label='Total ')
    ax.plot(bin_centers, weights['sig'](bin_centers).flatten()/sum(weights['total'](bin_centers))*len(data[decay]), color='blue', label='Signal')
    ax.plot(bin_centers, weights['misid'](bin_centers).flatten()/sum(weights['total'](bin_centers))*len(data[decay]), color='yellowgreen', label='MisID')
    ax.plot(bin_centers, weights['comb'](bin_centers).flatten()/sum(weights['total'](bin_centers))*len(data[decay]), color='violet', label='Combinatorial')
    ax.plot(bin_centers, weights['low'](bin_centers).flatten()/sum(weights['total'](bin_centers))*len(data[decay]), color='pink', label='Low')
    ax.legend()

    ax.set_xlabel('$M(B^{\pm})$')


In [ ]:
print(x_exp)

In [ ]:
fig, ax = plt.subplots()
plotOn(data=Bu_M, var=[], decay='b2dpi_DD',  ax=ax, nbins=144)

# Simultaneous fit on DK and DPi

In [49]:
def nll_extended_dk_dd(par):
    decay = 'b2dk_DD'
    x= Bu_M[decay]
    nsig = par[0]
    nmisid = par[3]/(varDict_init['pideff_DPi_KsPiPi_p_to_p_DD']) * varDict_init['pideff_DPi_KsPiPi_p_to_k_DD']
    ncomb = par[1]
    nlow = par[2]
    frac_low_misID = varDict_init['cp_range_n_low_misID_DK_KsPiPi_DD']/varDict_init['cp_range_n_low_DPi_KsPiPi_DD']
    nlow_misID = par[5]*frac_low_misID
    nlow_Bs2DKPi = varDict_init['cp_range_n_low_Bs2DKPi_DK_KsPiPi_DD']

    pdf = lambda x: pdfs_data[decay]['sig'](x)*nsig
    pdf2 = lambda x: pdfs_data[decay]['misid'](x)*nmisid
    pdf3 = lambda x: pdfs_data[decay]['comb'](x)*ncomb
    pdf4 = lambda x: pdfs_data[decay]['low'](x)*nlow
    pdf5 = lambda x: pdfs_data[decay]['low_misID'](x)*nlow_misID
    pdf6 = lambda x: pdfs_data[decay]['low_Bs2DKPi'](x)*nlow_Bs2DKPi
    ntotal = (nsig + nmisid + ncomb + nlow + nlow_misID + nlow_Bs2DKPi)

    npdf = pdf(x) + pdf2(x) + pdf3(x) + pdf4(x) + pdf5(x) + pdf6(x)


    nll =  tf.reduce_sum(- 2 *clip_log(npdf)) + 2* ntotal
    
    return nll

def nll_extended_dpi_dd(par):
    decay = 'b2dpi_DD'
    x= Bu_M[decay]
    nsig = par[3]
    nmisid = par[0]/(varDict_init['pideff_DK_KsPiPi_k_to_k_DD']) * varDict_init['pideff_DK_KsPiPi_k_to_p_DD']
    ncomb = par[4]
    nlow = par[5]

    pdf = lambda x: pdfs_data[decay]['sig'](x)*nsig
    pdf2 = lambda x: pdfs_data[decay]['misid'](x)*nmisid
    pdf3 = lambda x: pdfs_data[decay]['comb'](x)*ncomb
    pdf4 = lambda x: pdfs_data[decay]['low'](x)*nlow

    ntotal = (nsig + nmisid + ncomb + nlow )

    npdf = pdf(x) + pdf2(x) + pdf3(x) + pdf4(x)

    nll = tf.reduce_sum(- 2 * clip_log(npdf)) + 2 * ntotal
    
    return nll   

def nll_dd(x):
    print(nll_extended_dk_dd(x) + nll_extended_dpi_dd(x))
    return nll_extended_dk_dd(x) + nll_extended_dpi_dd(x)

In [50]:
x_ini = [varDict_init['cp_range_n_sig_DK_KsPiPi_DD'], varDict_init['cp_range_n_comb_DK_KsPiPi_DD'], varDict_init['cp_range_n_low_DK_KsPiPi_DD'],  varDict_init['cp_range_n_sig_DPi_KsPiPi_DD'], varDict_init['cp_range_n_comb_DPi_KsPiPi_DD'], varDict_init['cp_range_n_low_DPi_KsPiPi_DD']]

n_data = {'b2dk_DD': len(Bu_M['b2dk_DD']), 'b2dpi_DD': len(Bu_M['b2dpi_DD'])}
bounds = [(-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD']), (-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD']), (-n_data['b2dk_DD'], 1.2*n_data['b2dk_DD']), (-n_data['b2dpi_DD'], 1.2*n_data['b2dpi_DD']), (-n_data['b2dpi_DD'], 1.2*n_data['b2dpi_DD']), (-n_data['b2dpi_DD'], 1.2*n_data['b2dpi_DD'])]
model = minimize(nll_dd, x_ini, method="L-BFGS-B", bounds=bounds)


tf.Tensor(-2096640.7038810372, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810402, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810374, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810372, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810377, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810377, shape=(), dtype=float64)
tf.Tensor(-2096640.7038810377, shape=(), dtype=float64)
tf.Tensor(-2096640.775786291, shape=(), dtype=float64)
tf.Tensor(-2096640.7757862932, shape=(), dtype=float64)
tf.Tensor(-2096640.7757862916, shape=(), dtype=float64)
tf.Tensor(-2096640.7757862913, shape=(), dtype=float64)
tf.Tensor(-2096640.775786291, shape=(), dtype=float64)
tf.Tensor(-2096640.775786291, shape=(), dtype=float64)
tf.Tensor(-2096640.775786291, shape=(), dtype=float64)
tf.Tensor(-2096640.9883166063, shape=(), dtype=float64)
tf.Tensor(-2096640.9883166088, shape=(), dtype=float64)
tf.Tensor(-2096640.9883166065, shape=(), dtype=float64)
tf.Tensor(-2096640.9883166063, shape=(), dtype=float

In [51]:
minimized_dk_dd_nsig = model.x[0]
minimized_dk_dd_nmisid = model.x[3]/(varDict_init['pideff_DPi_KsPiPi_p_to_p_DD']) * varDict_init['pideff_DPi_KsPiPi_p_to_k_DD']
minimized_dk_dd_ncomb = model.x[1]
minimized_dk_dd_nlow = model.x[2]
dk_dd_frac_low_misID = varDict_init['cp_range_n_low_misID_DK_KsPiPi_DD']/varDict_init['cp_range_n_low_DPi_KsPiPi_DD']
minimized_dk_dd_nlow_misID = model.x[5]*dk_dd_frac_low_misID
minimized_dk_dd_nlow_Bs2DKPi = varDict_init['cp_range_n_low_Bs2DKPi_DK_KsPiPi_DD']
minimized_dpi_dd_nsig = model.x[3]
minimized_dpi_dd_nmisid = model.x[0]/(varDict_init['pideff_DK_KsPiPi_k_to_k_DD']) * varDict_init['pideff_DK_KsPiPi_k_to_p_DD']
minimized_dpi_dd_ncomb = model.x[4]
minimized_dpi_dd_nlow = model.x[5]

n_obs = [minimized_dk_dd_nsig, minimized_dk_dd_nmisid, minimized_dk_dd_ncomb, minimized_dk_dd_nlow, minimized_dk_dd_nlow_misID, minimized_dk_dd_nlow_Bs2DKPi, minimized_dpi_dd_nsig, minimized_dpi_dd_nmisid, minimized_dpi_dd_ncomb, minimized_dpi_dd_nlow]


In [52]:
x_exp = [varDict['n_sig_DK_KsPiPi_DD'], varDict['n_misid_DK_KsPiPi_DD'], varDict['n_comb_DK_KsPiPi_DD'], varDict['n_low_DK_KsPiPi_DD'], varDict['n_low_misID_DK_KsPiPi_DD'], varDict['n_low_Bs2DKPi_DK_KsPiPi_DD'], varDict['n_sig_DPi_KsPiPi_DD'], varDict['n_misid_DPi_KsPiPi_DD'], varDict['n_comb_DPi_KsPiPi_DD'], varDict['n_low_DPi_KsPiPi_DD']]

In [53]:
#Difference
print('nsig: %.2f'%(n_obs[0] - x_exp[0]))
print('nmisid: %.2f'%(n_obs[1] - x_exp[1]))
print('ncomb: %.2f'%(n_obs[2] - x_exp[2]))
print('nlow: %.2f'%(n_obs[3] - x_exp[3]))
print('nlow_misID: %.2f'%(n_obs[4] - x_exp[4]))
print('nlow_Bs2DKPi: %.2f'%(n_obs[5] - x_exp[5]))
print('nsig: %.2f'%(n_obs[6] - x_exp[6]))
print('nmisid: %.2f'%(n_obs[7] - x_exp[7]))
print('ncomb: %.2f'%(n_obs[8] - x_exp[8]))
print('nlow: %.2f'%(n_obs[9] - x_exp[9]))

nsig: 112.31
nmisid: 0.86
ncomb: -154.13
nlow: 127.48
nlow_misID: -29.84
nlow_Bs2DKPi: 0.08
nsig: 29.43
nmisid: 16.93
ncomb: -208.46
nlow: 110.97
